* 输入
    * feather数据文件
    * 模型结构和参数
    * 模型权值文件
* 输出
    * 评估标准

In [ ]:
# import comet_ml in the top of your file
from comet_ml import Experiment

# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="Go4XzK6f4G10W9jFaWiHg6JP2",
                        project_name="general", workspace="twofyw")

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
context = Context('validation', debug=False, sl=30, cont_names=Context.all_columns(), postpond=0, fn_feather='tbmData/feather/all_columns-full-30-s-0.feather')
databunch = MultiDeptTabularDataBunch.from_df('tmp', config.cyc_cont, config.dep_var, valid_idx=config.valid_idx_tile, bs=config.bs, num_workers=0)

torch.cuda.set_device(config.gpu_start); torch.ones((1, 1)).cuda(); # wakeup gpu
record = False

# Task 1

## RNN regression

In [ ]:
from mymodels import *

In [ ]:
hyper_params = {
    'n_hidden': 100,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.6,
    'weight_p': 0.5,
    'layers': [3*config.n_cont, 5, 2],
    'drops': [0.2] * 3,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
#     'loss_func' = MSELossFlat()
    'loss_func': l1
}

learner = get_cont_model(context, databunch, hyper_params)

In [ ]:
learner.predict??

In [ ]:
learner.load('rnn-normalized-full-30-s-0-1');

In [ ]:
def validate(learner):
    xb,yb = learner.data.one_batch(DatasetType.Valid, detach=False, denorm=False)
    # remove noise
    learner.data.
    
    yb = to_detach(yb)
    cb_handler = CallbackHandler(learner.callbacks) # rnn trainer
    preds, _ = loss_batch(learner.model.eval(), xb, yb, cb_handler=cb_handler)
    loss, mapd = learner.loss_func(preds, yb), learner.metrics[0](preds, yb)
    
    # Plot p against y. Expect linearity
    yb_denormed, pb_denormed = denormalize(y_np, *stat_y), denormalize(p_np, *stat_y)
    
    
    return loss, mapd


In [ ]:
i, j = 0, 1
x, y = learner.data.train_ds[i]
xlast = x
x, y = learner.data.train_ds[i]
torch.all(x.data[1] == xlast.data[1])

In [ ]:
loss, m = validate(learner)

In [ ]:
xb,yb = learner.data.one_batch(DatasetType.Valid, detach=False, denorm=False)

In [ ]:
it = iter(learner.data.valid_dl)